In [18]:
%reload_ext autoreload
%autoreload 2
import sys; sys.path.append('..')
from functools import partial
from src.data_utils import get_dataset_imagenet
from jupyter_preproc import smoothen, probs_to_softacc, nonmono, thresh, easy, normalize, get_entropy, accuracy
import numpy as np
import time
import os

rng = np.random.default_rng(5)

In [19]:
tname = 'imagenet'
dnames = ['imagenet_val', 'imagenet_sketch']
gnames = ['resnet50']

# smoothing parameters
num_gridpts = 50
smooth = True
sigma = 2.0

xkey = 'x'
Ykey = 'probs'

measures = {'ent': get_entropy}
funcs = [nonmono, thresh, easy]

for dname in dnames:
    dataset = get_dataset_imagenet(dname)
    ytest = dataset.targets
    measures['acc'] = partial(accuracy, ytest=ytest)
    
    for gname in gnames:
        print(f'{dname} {gname}')
        fname = f'../processed/{tname}/{gname}/{gname}_{dname}.npz'
        
        print('loading processed')
        G = np.load(fname)
        print('done\n')
        
        # G['xsoft'] = probs_to_softacc(G['probs'], ytest)
        
        print('smoothing')
        res = smoothen(G, xkey, Ykey)
        print('done\n')
        
        scores = dict()
        for meas_name, meas in measures.items():
            print(meas_name)
            m = meas(res[f'{Ykey}_smooth'])
            print(m.shape)
    
            for func in funcs:
                print(func.__name__)
                f_name = f'{meas_name}_{func.__name__}'
                s_vec = func(m)
                print(s_vec.shape)
                scores[f_name] = normalize(s_vec)
        
        os.makedirs(f'../scores/{tname}/{gname}/', exist_ok=True)
        outfname = f'../scores/{tname}/{gname}/{gname}_{dname}_scores.npz'
        
        print(f'saving to: {outfname}')                     
        with open(outfname, 'wb') as f:
            np.savez(f, **scores)
        print('done\n')

imagenet_val resnet50
loading processed
done

smoothing
done

ent
(50000, 50)
nonmono
(50000,)
thresh
(50000,)
easy
(50000,)
acc
(50000, 50)
nonmono
(50000,)
thresh
(50000,)
easy
(50000,)
saving to: ../scores/imagenet/resnet50/resnet50_imagenet_val_scores.npz
done

imagenet_sketch resnet50
loading processed
done

smoothing
done

ent
(50889, 50)
nonmono
(50889,)
thresh
(50889,)
easy
(50889,)
acc
(50889, 50)
nonmono
(50889,)
thresh
(50889,)
easy
(50889,)
saving to: ../scores/imagenet/resnet50/resnet50_imagenet_sketch_scores.npz
done

